In [16]:
#Adjustments AI 
#Sample data creator


In [14]:
#Shared common config
SpecialFields=["AdjustmentType","AdjustmentName","AdjustedField"]
AuditFields=["OldVal","NewVal"]
SampleFields=["CustomerID","CustomerType","RiskCategory","IsDefaulted","CountryCode"]
NumericFields=["Drawn","Undrawn"]
RatioFields=["PD"]
AdjustmentTypes=["Amend"]
AdjustmentNames=["GRC_EndMonth","Fix_Broken_Batch","Credit_Uplift","Defaulting_Reclassify","Default"]
AdjustedFields=SampleFields
CustomerID=["CUST238947","CUST563201","CUST981372","CUST450896","CUST712035","CUST103289","CUST665428","CUST359147","CUST804231","CUST192654"]
CustomerType=["Retail","Corporate","SME"]
RiskCategory=["Low","Medium","High"]
IsDefaulted=["0","1"]
CountryCode=["US","GB","DE"]
FieldMapping={"CustomerID":CustomerID,"CustomerType":CustomerType,"RiskCategory":RiskCategory,"IsDefaulted":IsDefaulted,"CountryCode":CountryCode}

#Define adjustment rules
AdjustmentRules=[]
AdjustmentRules.append([[{"CustomerType":"Corporate"},{"RiskCategory":"High"}],["PD",0.75,"Market Downturn"]])
AdjustmentRules.append([[{"RiskCategory":"High"},{"IsDefaulted":"1"}],["CustomerType","Debt Recovery","Defaulting Cleanup"]])


In [15]:
#Generate random data with Adjustment rules applied
import pandas as pd
import numpy as np

#Sample Data Generator
records=1000

#Hide warmings from pandas
pd.options.mode.chained_assignment = None  # default='warn'

def GetRandomFieldValue(field):
    return np.random.choice(FieldMapping[field],1)[0]

#Checks if this row matches any adjustment rule, if so returns the [{fieldToUpdate:NewValue},AdjustmentComment]
def ValidateGetAdjustmentRule(row):
    for rule in AdjustmentRules:
        criteria=rule[0]
        adjustment=rule[1]

        #Check all criteria match
        allMatch=True
        for crit in criteria:
            #Split crit dictionary into key and value
            critField=list(crit.keys())[0]
            critValue=crit[critField]

            #Check if row matched
            if row[critField]!=critValue:
                allMatch=False
        
        if allMatch:
            return allMatch,adjustment
    
    return False,[]


df=pd.DataFrame(columns=SampleFields+SpecialFields+AuditFields)
for mapping in FieldMapping:
    df[mapping]=np.random.choice(FieldMapping[mapping],records)
df["AdjustmentType"]=np.random.choice(AdjustmentTypes,records)
df["AdjustmentName"]=np.random.choice(AdjustmentNames,records)

#Adjusted field------------------------------------------------------------------------
df["AdjustedField"]=np.random.choice(AdjustedFields,records)
#df["AdjustedField"]="PD"

for field in NumericFields:
    #Insert new column after first 1
    df[field]=np.random.randint(1000000, size=records)
for field in RatioFields:
    df[field]=np.random.randint(100, size=records)/100

#Add in audit fields
df["OldVal"]=np.nan
df["NewVal"]=np.nan

#Loop over each row and generate a new value for Oldval based on field
for index, row in df.iterrows():
    adjField=row["AdjustedField"]

    #Check if this row matches any adjustment rule
    matchesRule,rule = ValidateGetAdjustmentRule(row)

    #APPLY ADJUSTMENTS--------------------------------------
    if adjField not in NumericFields and adjField not in RatioFields:

        if matchesRule:
            
            #Split rule dictionary into key and value
            ruleTarget=rule[0]
            ruleValue=rule[1]
            ruleComment=rule[2]
            df["AdjustmentName"][index]=ruleComment
            df["AdjustedField"][index]=ruleTarget
            df["NewVal"][index]=ruleValue
        else:
            #Random "noise" adjustments
            #df["OldVal"][index]=row[adjField]
            df["NewVal"][index]=GetRandomFieldValue(adjField)

# #Reshuffle the numeric fields to the start
df=df.reindex(columns=SpecialFields+NumericFields+RatioFields+SampleFields+AuditFields)

#Remove OldVal Field
df=df.drop(columns=["OldVal"])

# #Write to CSV file
df.to_csv("SampleData.csv",index=False)
df


,AdjustmentType,AdjustmentName,AdjustedField,Drawn,Undrawn,PD,CustomerID,CustomerType,RiskCategory,IsDefaulted,CountryCode,NewVal
0,Amend,Credit_Uplift,CountryCode,411008,696860,0.12,CUST238947,Corporate,Medium,1,US,DE
1,Amend,GRC_EndMonth,CustomerID,238950,391926,0.06,CUST665428,SME,Medium,0,GB,CUST563201
2,Amend,Defaulting_Reclassify,IsDefaulted,529405,534799,0.77,CUST981372,Retail,High,0,DE,0
3,Amend,Fix_Broken_Batch,CustomerType,776085,123677,0.69,CUST103289,Retail,High,1,US,SME
4,Amend,Market Downturn,PD,952813,259908,0.40,CUST981372,Corporate,High,0,US,0.75
...,...,...,...,...,...,...,...,...,...,...,...,...
995,Amend,GRC_EndMonth,CustomerType,878749,546459,0.32,CUST563201,SME,Low,1,GB,Corporate
996,Amend,GRC_EndMonth,CountryCode,625047,711819,0.27,CUST450896,SME,High,1,US,GB
997,Amend,Default,CountryCode,202963,328738,0.13,CUST450896,Corporate,Low,0,DE,US
998,Amend,Credit_Uplift,CustomerType,80742,719180,0.87,CUST103289,Retail,Low,0,GB,SME


3

In [4]:
len(df["NewVal"].unique())

22